In [10]:
!pip install openai==1.82.0

In [11]:
import os
from openai import OpenAI

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 OpenAI 客户端（假设 DeepSeek 的 API 兼容 OpenAI 格式）
client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

# 定义提示词
prompt = """请用 HTML5 生成一个完整的五子棋游戏，要求：

## 严格要求

1. **输出格式**：
   - 直接输出纯 HTML 代码，不要添加 ```html 或 ``` 标记
   - 从 <!DOCTYPE html> 开始
   - 到 </html> 结束
   - 中间不要有任何 markdown 代码块标记

2. **编码问题 - 非常关键**：
   - 所有中文字符必须使用 UTF-8 编码
   - 不要使用 HTML 实体编码（如 &#20116; 这样的编码）
   - 必须包含 <meta charset="UTF-8"> 标签
   - 游戏状态文本示例："黑方回合"、"白方获胜"、"黑方获胜"、"平局"

3. **棋盘绘制 - Canvas 优化**：
   - Canvas 宽高都设置为 600px
   - 网格线宽度必须设置为 2px（ctx.lineWidth = 2）
   - 棋盘背景色 #dcb35c
   - 网格线颜色 #000000
   - 星点（天元）半径为 4px

4. **棋子绘制 - 必须正确**：
   - 黑棋：fillStyle = '#000000'（纯黑色）
   - 白棋：fillStyle = '#FFFFFF'（纯白色），边框 #000000
   - 棋子半径 = 格子大小 * 0.4
   - 黑棋可添加小高光（透明白色），白棋可添加边框

5. **游戏逻辑 - 必须正确**：
   - 黑方先行（player = 1）
   - 白方后行（player = 2）
   - 获胜判断：当前落子后，检查该位置四个方向（水平、垂直、两条对角线）
   - 每个方向检查：当前位置 + 正向4个位置 + 反向4个位置 = 最多9个位置
   - 如果任意方向连成5个或以上同色棋子，则当前玩家获胜
   - 必须正确判断黑棋获胜和白棋获胜，不能混淆

6. **状态显示**：
   - 显示当前玩家（"黑方回合" 或 "白方回合"）
   - 游戏结束显示准确的胜者（"黑方获胜" 或 "白方获胜"）
   - 如果棋盘满了就平局（"平局"）

7. **功能需求**：
   - 点击棋盘放置棋子
   - 悔棋按钮：撤销最后一步
   - 重新开始按钮：重置游戏
   - 棋盘大小 15×15
   - 星点位置：[3,3]、[3,11]、[7,7]、[11,3]、[11,11]

8. **代码结构**：
   - 所有 CSS 必须在 <style> 标签内（内联样式）
   - 所有 JavaScript 必须在 <script> 标签内
   - 使用 DOMContentLoaded 事件确保 DOM 加载完毕
   - 清晰的函数划分：initBoard()、drawPiece()、checkWin()、placePiece()、undoMove()、restartGame()

## 特别提醒

- 确保 checkWin() 函数逻辑清晰，不要混淆黑白棋的胜利判断
- 不要硬编码玩家名字，使用 player 变量来判断
- 按钮事件监听必须绑定正确的处理函数
- Canvas 点击坐标计算要准确

请生成完整可运行的 HTML 文件。"""

In [ ]:
try:
    # 调用 DeepSeek Chat API
    response = client.chat.completions.create(
        model="deepseek-chat",  # 或 DeepSeek 提供的其他模型名称
        messages=[
            {"role": "system", "content": "你是一个专业的 Web 开发助手。你必须生成完全正确的、可以在浏览器中运行的 HTML 代码。特别注意：\n1. 直接输出纯 HTML，不要加代码块标记\n2. 中文字符必须使用 UTF-8 编码，不要用 HTML 实体\n3. 游戏逻辑必须正确，黑白棋判胜不能混淆\n4. Canvas 绘制必须清晰"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        stream=False
    )

    # 提取生成的 HTML 内容
    if response.choices and len(response.choices) > 0:
        html_content = response.choices[0].message.content
        
        # 保存到文件
        with open("gomoku-3.html", "w", encoding="utf-8") as f:
            f.write(html_content)
        print("五子棋游戏已保存为 gomoku-3.html")
    else:
        print("未收到有效响应")

except Exception as e:
    print(f"调用 API 出错: {e}")